In [1]:
import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

In [2]:
input_ = pd.read_excel('./part.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
input_['UAC_Part_Number'].tolist()

总数量：300


['CO 0023GLC',
 'CO 0024GLC',
 'CO 0040C',
 'CO 0041C',
 'CO 0042C',
 'CO 0043C',
 'CO 0071C',
 'CO 0073C',
 'CO 0074C',
 'CO 0076C',
 'CO 04080C',
 'CO 10541AC',
 'CO 1100JC',
 'CO 1102JC',
 'CO 11136N',
 'CO 11282C',
 'CO 11444V',
 'CO 11446V',
 'CO 11460V',
 'CO 11463V',
 'CO 11513C',
 'CO 11514C',
 'CO 11516C',
 'CO 11517C',
 'CO 11539C',
 'CO 1206JC',
 'CO 1214',
 'CO 1214C',
 'CO 1222DC',
 'CO 1233JC',
 'CO 2011V',
 'CO 29264C',
 'CO 3057AC',
 'CO 3407AC',
 'CO 3605AC',
 'CO 4036C',
 'CO 4056V',
 'CO 4070C',
 'CO 4078C',
 'CO 4079',
 'CO 4081C',
 'CO 4084C',
 'CO 4095C',
 'CO 4096C',
 'CO 4115V',
 'CO 42021V',
 'CO 42031V',
 'CO 42038V',
 'CO 42070V',
 'CO 42120V',
 'CO 4281C',
 'CO 4301',
 'CO 4301C',
 'CO 4302C',
 'CO 4305C',
 'CO 4307C',
 'CO 4308C',
 'CO 4314C',
 'CO 4318C',
 'CO 4326',
 'CO 4326DC',
 'CO 4328C',
 'CO 4332C',
 'CO 4335',
 'CO 4335C',
 'CO 4336C',
 'CO 4347',
 'CO 4347C',
 'CO 4356C',
 'CO 4359C',
 'CO 4362C',
 'CO 4369C',
 'CO 4375C',
 'CO 4378C',
 'CO 4384C'

In [3]:
crawler_status = 'error'

print(crawler_status)

error


In [4]:
import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

In [5]:
a = 0

resp = requests.get('https://uacparts.com/Catalog/Buyers?part=' + input_.loc[a, 'UAC_Part_Number'],
                    headers=UA.get_User_Agent_Requests(),
                    proxies=Proxy.get_Proxy_Requests()).text

resp

'\r\n<div><br /></div>\r\n<form action="/Catalog/Buyers" method="post"><div>\r\n    <table class="awe-grid table">\r\n            <tr id=\'xrefCO 0023GLC\' class="awe-row">\r\n                \r\n                    <td class="awe-grid td">1980</td>\r\n                \r\n                <td class="awe-grid td">American Motors</td>\r\n                <td class="awe-grid td">Pacer</td>\r\n                <td class="awe-grid td">Limited</td>\r\n                <td class="awe-grid td">L6 CC:- CID:258 4.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:C</td>\r\n            </tr>\r\n            <tr id=\'xrefCO 0023GLC\' class="awe-row">\r\n                \r\n                    <td class="awe-grid td">1980</td>\r\n                \r\n                <td class="awe-grid td">American Motors</td>\r\n                <td class="awe-grid td">Pacer</td>\r\n                <td class="awe-grid td">DL</td>\r\n                <td class="awe-grid td">L6 CC:- CID:258 4.2L OHV CARB 2BBL Naturally Aspirated 

In [6]:
from bs4 import BeautifulSoup
from lxml import etree

In [7]:
soup = BeautifulSoup(resp.strip(), 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

html

<html>
 <body>
  <div>
   <br/>
  </div>
  <form action="/Catalog/Buyers" method="post">
   <div>
    <table class="awe-grid table">
     <tr class="awe-row" id="xrefCO 0023GLC">
      <td class="awe-grid td">
       1980
      </td>
      <td class="awe-grid td">
       American Motors
      </td>
      <td class="awe-grid td">
       Pacer
      </td>
      <td class="awe-grid td">
       Limited
      </td>
      <td class="awe-grid td">
       L6 CC:- CID:258 4.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:C
      </td>
     </tr>
     <tr class="awe-row" id="xrefCO 0023GLC">
      <td class="awe-grid td">
       1980
      </td>
      <td class="awe-grid td">
       American Motors
      </td>
      <td class="awe-grid td">
       Pacer
      </td>
      <td class="awe-grid td">
       DL
      </td>
      <td class="awe-grid td">
       L6 CC:- CID:258 4.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:C
      </td>
     </tr>
     <tr class="awe-row" id="xrefCO 0023GLC">
      <td cl

<Element html at 0x11f6677c0>

In [8]:
dict_vehicle = {}
list_tr = html.xpath('//table/tr')
for tr in list_tr:
    list_td = tr.xpath('./td')
    make = '' if list_td[1].text == None else list_td[1].text.strip()
    model = '' if list_td[2].text == None else list_td[2].text.strip()
    engine = '' if list_td[4].text == None else list_td[4].text.strip()
    year = int(list_td[0].text.strip())
    
    if make not in dict_vehicle:
        dict_vehicle[make] = {model: {engine: [year]}}
    else:
        if model not in dict_vehicle[make]:
            dict_vehicle[make][model] = {engine: [year]}
        else:
            if engine not in dict_vehicle[make][model]:
                dict_vehicle[make][model][engine] = [year]
            else:
                dict_vehicle[make][model][engine] += [year]
                
    dict_vehicle[make][model][engine] = sorted(list(set(dict_vehicle[make][model][engine])))
    
dict_vehicle

{'American Motors': {'Pacer': {'L6 CC:- CID:258 4.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:C': [1980]}},
 'Compressors and Components': {'Compressors': {'': [1]}},
 'Dodge': {'Diplomat': {'L6 CC:3687 CID:225 3.7L OHV CARB 1BBL Naturally Aspirated GAS VIN:H': [1983],
   'V8 CC:5211 CID:318 5.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:P': [1983],
   'V8 CC:5211 CID:318 5.2L OHV CARB 4BBL Naturally Aspirated GAS VIN:R': [1983],
   'L6 CC:3687 CID:225 3.7L OHV CARB 1BBL Naturally Aspirated GAS VIN:J': [1983],
   'V8 CC:5211 CID:318 5.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:4': [1983],
   'V8 CC:5211 CID:318 5.2L OHV CARB 4BBL Naturally Aspirated GAS VIN:S': [1983],
   'L6 CC:3687 CID:225 3.7L U/K CARB U/K Naturally Aspirated GAS VIN:K': [1983],
   'V8 CC:5211 CID:318 5.2L U/K CARB U/K Naturally Aspirated GAS VIN:Z': [1983],
   'L6 CC:3687 CID:225 3.7L OHV CARB 1BBL Naturally Aspirated GAS VIN:E': [1981,
    1982],
   'V8 CC:5211 CID:318 5.2L OHV CARB 2BBL Naturally Aspirated GAS

In [9]:
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        list_year = []
        for engine in dict_vehicle[make][model]:
            list_year += dict_vehicle[make][model][engine]
        list_year = sorted(list(set(list_year)))
        
        year_min = min(list_year)
        year_max = max(list_year)
        if year_min == year_max:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
        else:
            list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

vehicle_1 = '\n'.join(sorted(list_vehicle))

print(vehicle_1)

American Motors Pacer 1980
Compressors and Components Compressors 1
Dodge Diplomat 1980-1983
FREIGHTLINER ANY 1
Ford Bronco 1973-1985
Ford Bronco II 1984
Ford E-100 Econoline 1973-1983
Ford E-100 Econoline Club Wagon 1983
Ford E-150 Econoline 1983
Ford E-150 Econoline Club Wagon 1983
Ford Thunderbird 1985-1988
Volvo 242 1975-1984
Volvo 244 1975-1984
Volvo 245 1976-1984


In [10]:
list_vehicle = []
for make in dict_vehicle:
    for model in dict_vehicle[make]:
        for engine in dict_vehicle[make][model]:
            year_min = min(dict_vehicle[make][model][engine])
            year_max = max(dict_vehicle[make][model][engine])
            if year_min == year_max:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
            else:
                if engine == '':
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                else:
                    list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

vehicle_2 = '\n'.join(sorted(list_vehicle))

print(vehicle_2)

American Motors Pacer 1980 L6 CC:- CID:258 4.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:C
Compressors and Components Compressors 1
Dodge Diplomat 1980 L6 CC:3687 CID:225 3.7L OHV CARB 1BBL Naturally Aspirated GAS VIN:C
Dodge Diplomat 1980 L6 CC:3687 CID:225 3.7L OHV CARB 2BBL Naturally Aspirated GAS VIN:D
Dodge Diplomat 1980 V8 CC:5211 CID:318 5.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:G
Dodge Diplomat 1980 V8 CC:5211 CID:318 5.2L OHV CARB 4BBL Naturally Aspirated GAS VIN:H
Dodge Diplomat 1981 L6 CC:3687 CID:225 3.7L U/K CARB U/K Naturally Aspirated GAS VIN:G
Dodge Diplomat 1981-1982 L6 CC:3687 CID:225 3.7L OHV CARB 1BBL Naturally Aspirated GAS VIN:E
Dodge Diplomat 1981-1982 L6 CC:3687 CID:225 3.7L OHV CARB 1BBL Naturally Aspirated GAS VIN:F
Dodge Diplomat 1981-1982 V8 CC:5211 CID:318 5.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:K
Dodge Diplomat 1981-1982 V8 CC:5211 CID:318 5.2L OHV CARB 2BBL Naturally Aspirated GAS VIN:L
Dodge Diplomat 1981-1982 V8 CC:5211 CID:318 5.2L OHV CAR

In [11]:
df_temp = pd.DataFrame([{'UAC_Part_Number': input_.loc[a, 'UAC_Part_Number'],
                         'UAC_Vehicle_1': vehicle_1,
                         'UAC_Vehicle_2': vehicle_2}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

len(output_correct)

1

In [12]:
crawler_status = 'ok'

crawler_status

'ok'

In [13]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'UAC_Part_Number': input_.loc[a, 'UAC_Part_Number']}])
    
    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

len(output_error)

0

In [14]:
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['UAC_Part_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./test_uac_vehicle.xlsx', index=False)

output_correct

,UAC_Part_Number,UAC_Vehicle,UAC_Vehicle_2
0,CO 0023GLC,American Motors Pacer 1980\nCompressors and Co...,American Motors Pacer 1980 L6 CC:- CID:258 4.2...


In [15]:
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['UAC_Part_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./test_uac_vehicle_error.xlsx', index=False)

output_error

""
